In [1]:
# Import some useful libraries
%matplotlib inline
import pandas as pd
import urllib
import xml.etree.ElementTree as ET
import numpy as np
import nltk
import rake
from bs4 import BeautifulSoup as BSoup
import nlp_helpers
%load_ext autoreload
%autoreload 2

# display all pandas columns
pd.set_option('display.max_columns', 100)

# NLP content extraction with GENSIM

First attempt of transcript content extraction
(Tokenization|Parser|Text Segmentation|Word Break rules|Text Analysis)

http://gerardnico.com/wiki/natural_language/parser


**Tested: **
* RAKE: good but basic, needs a lot of by-hand process (stemming, ...) but perfectly controlled so we can control everything
* TextRank: in python2... too lazy to change my path ^^
* KEA: Do not seem to be still maintained 
* MAUI: Improvement of KEA, works with supervised learning and controlled vocabulary. Works with java

## Gensim

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

* NLTK, a natural language toolkit for Python. A useful package for any natural language processing.
    * For Mac/Unix with pip: $ sudo pip install -U nltk.
    
* stop_words, a Python package containing stop words.
    * For Mac/Unix with pip: $ sudo pip install stop-words.
    
* gensim, a topic modeling package containing our LDA model.
    * For Mac/Unix with pip: $ sudo pip install gensim.
    
* clips.pattern for gensim lemmatization
    * -_- only python2 !! If really usefull, I'll do some trick to adapt it

### CSV load

In [2]:
df_transcripts = pd.read_csv("../scraping/data/Transcript.csv", nrows=50000)
df_transcripts = df_transcripts.loc[df_transcripts['LanguageOfText'] == 'FR']
df_transcripts.head(2)

,Unnamed: 0,CantonAbbreviation,CantonId,CantonName,CouncilId,CouncilName,DisplaySpeaker,End,Function,ID,IdSession,IdSubject,Language,LanguageOfText,MeetingCouncilAbbreviation,MeetingDate,MeetingVerbalixOid,Modified,ParlGroupAbbreviation,ParlGroupName,PersonNumber,SortOrder,SpeakerFirstName,SpeakerFullName,SpeakerLastName,Start,Text,Type,VoteId
0,0,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:50:21.663,NaN,1,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.91,C,Groupe démocrate-chrétien,498.0,3,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:36:50.007,<pd_text><p>La séance d'aujourd'hui s'ouvre so...,1,NaN
2,2,VD,22.0,Vaud,1.0,Conseil national,True,1999-12-06T14:33:39.803,NaN,3,4601,1,FR,FR,N,19991206,637,2015-11-09T11:48:16.937,C,Groupe démocrate-chrétien,498.0,1,Jacques,Neirynck Jacques,Neirynck,1999-12-06T14:32:24.303,<pd_text><p>Madame la Présidente de la Confédé...,1,NaN


In [23]:
df_transcripts.shape

(10597, 29)

### Clean Up pipeline

* HTML tags
* lower cases
* tokenization
* removing stop words
* stemming

In [18]:
text = df_transcripts['Text']
clean_text = nlp_helpers.cleaning_pipeline(text)
clean_text.head(10)

0     [séanc, aujourd, ouvr, sign, hasard, dont, con...
2     [madam, président, confédérat, sehr, geehrt, f...
11    [sembl, acqui, majorité, conseil, nation, élir...
12    [ainsi, rotat, immuabl, présidenc, voudrait, a...
13    [venez, recevoir, feuill, vert, donn, list, gr...
21    [malheureus, sort, certain, interpel, lesquel,...
27    [essentiel, conseil, fédéral, rallié, décision...
33    [pens, bien, commenc, législatur, thème, voir,...
38    [group, libéral, acceptera, arrêté, fédéral, c...
39    [aprè, démonstrat, spectaculair, cultur, popul...
Name: Text, dtype: object

In [19]:
from gensim import corpora, models

# construct a document-term matrix
dictionary = corpora.Dictionary(clean_text)

In [20]:
# converted into a bag-of-words:
corpus = [dictionary.doc2bow(text) for text in clean_text]

In [27]:
dictionary[0]

'inventeur'

In [21]:
import gensim
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word = dictionary, passes=5)

In [22]:
for i, bag in enumerate(ldamodel.print_topics(num_topics=20, num_words=8)):
    print("============")
    print("Cluster ", i, ": ", bag)

Cluster  0 :  (0, '0.019*"accord" + 0.015*"schengen" + 0.014*"nicht" + 0.014*"union" + 0.014*"européenn" + 0.012*"dass" + 0.009*"ein" + 0.008*"auch"')
Cluster  1 :  (1, '0.015*"canton" + 0.012*"travail" + 0.012*"plu" + 0.011*"assur" + 0.006*"prestat" + 0.006*"systèm" + 0.006*"mesur" + 0.006*"caiss"')
Cluster  2 :  (2, '0.018*"articl" + 0.017*"commiss" + 0.013*"conseil" + 0.012*"droit" + 0.009*"alinéa" + 0.008*"fédéral" + 0.007*"plu" + 0.006*"etat"')
Cluster  3 :  (3, '0.035*"proposit" + 0.028*"antrag" + 0.015*"commiss" + 0.015*"conseil" + 0.014*"stimmen" + 0.012*"président" + 0.012*"adopté" + 0.011*"minorité"')
Cluster  4 :  (4, '0.059*"initi" + 0.046*"group" + 0.020*"proposit" + 0.018*"majorité" + 0.016*"libéral" + 0.016*"projet" + 0.014*"canton" + 0.014*"populair"')
Cluster  5 :  (5, '0.028*"conseil" + 0.023*"commiss" + 0.020*"proposit" + 0.014*"fédéral" + 0.012*"minorité" + 0.011*"majorité" + 0.008*"plu" + 0.008*"articl"')
Cluster  6 :  (6, '0.020*"franc" + 0.014*"million" + 0.010*"